# Tree-based Methods
## Decision Trees, Random Forests, Gradient Boosted Decision Trees

The objective for this notebook is to take a high-level look at some of the tools available for tree-based models in scikit-learn

```
conda update scikit-learn
```

In [1]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# For producing decision tree diagrams.
from IPython.core.display import Image, display
from sklearn.externals.six import StringIO

# Data

Today we will use the boston housing data. Each row is a neighborhood. The features correspond to properties of houses in that neighborhood (e.g. average number of rooms). The target is the mean price (in \$1Ks) for that neighborhood

In [2]:
from sklearn.datasets import load_boston
boston = load_boston()

In [3]:
print boston.DESCR

Boston House Prices dataset

Notes
------
Data Set Characteristics:  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive
    
    :Median Value (attribute 14) is usually the target

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pupil-teacher ratio by town
      

### Load data into Pandas

In [8]:
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df['Price'] = boston.target
df.head

<bound method DataFrame.head of          CRIM    ZN  INDUS  CHAS    NOX     RM    AGE     DIS   RAD    TAX  \
0     0.00632  18.0   2.31   0.0  0.538  6.575   65.2  4.0900   1.0  296.0   
1     0.02731   0.0   7.07   0.0  0.469  6.421   78.9  4.9671   2.0  242.0   
2     0.02729   0.0   7.07   0.0  0.469  7.185   61.1  4.9671   2.0  242.0   
3     0.03237   0.0   2.18   0.0  0.458  6.998   45.8  6.0622   3.0  222.0   
4     0.06905   0.0   2.18   0.0  0.458  7.147   54.2  6.0622   3.0  222.0   
5     0.02985   0.0   2.18   0.0  0.458  6.430   58.7  6.0622   3.0  222.0   
6     0.08829  12.5   7.87   0.0  0.524  6.012   66.6  5.5605   5.0  311.0   
7     0.14455  12.5   7.87   0.0  0.524  6.172   96.1  5.9505   5.0  311.0   
8     0.21124  12.5   7.87   0.0  0.524  5.631  100.0  6.0821   5.0  311.0   
9     0.17004  12.5   7.87   0.0  0.524  6.004   85.9  6.5921   5.0  311.0   
10    0.22489  12.5   7.87   0.0  0.524  6.377   94.3  6.3467   5.0  311.0   
11    0.11747  12.5   7.87   0.0

### Keep track of predictors and target

In [10]:
predictors = boston.feature_names
target = 'Price'

In [11]:
predictors

array(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
       'TAX', 'PTRATIO', 'B', 'LSTAT'], 
      dtype='|S7')

### Train/Test Split

In [7]:
from sklearn.model_selection import train_test_split


# Decision Tree

A decision tree uses _recursive binary splits_ to segment the data into disjoint subsets. The splits are chosen according to _impurity_. For regression problems, impurity usually mean variance. For classification, impurity could mean entropy or gini impurity.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

Fit to the training data

### Visualize the tree

```
pip install pydotplus
brew install graphviz
```

In [ ]:
from IPython.display import Image  
from sklearn import tree
import pydotplus
dot_data = tree.export_graphviz(dt, out_file='tmp.dot',
                                feature_names=predictors,  
                                filled=True, rounded=True,  
                                special_characters=True)
graph = pydotplus.graphviz.graph_from_dot_file('tmp.dot')
Image(graph.create_png())  

### Accuracy on training data

Mean-squared error

mean((actual - pred) ** 2)

R-squared

### Exercise: Cross-validation Score

We would like to get a better estimate of the decision tree performance, but without contaminating the test data. Try using cross-validation to estimate the performance of the decision tree

In [ ]:
from sklearn.model_selection import cross_val_score

### Grid Search CV

Sklearn doesn't allow _post-pruning_ of decision trees, but we can try some pre-pruning parameters

In [ ]:
from sklearn.model_selection import GridSearchCV


## Random Forest

A random forest consists of N independently trained decision trees. The trees then vote on their final prediction

Random Forests are _very_ robust to parameter choices. Rules of thumb:
- Individual trees should be grown as deep as possible (lowering bias)
- Trees need to be _decorrelated_. Usually this is achieved just by bootstrapping (automatic) and choosing a subset of features at each split.
- You need 'enough' trees. 100-500 is probably fine

### Build a random forest and get cross-validated error on the training set

In [ ]:
from sklearn.ensemble import RandomForestRegressor

### Out-of-bag performance

Random forests come with a built in way to estimate performance on unseen data _without the need for cross-validation_.

Since each tree only sees a subset of the data (a bootstrap sample), each data point can be assigned an _oob prediction_ based only on the trees that it was not used to construct

In [ ]:
rf.fit(train_df[predictors], train_df[target])

In [ ]:
print "Train R^2: {:.3}".format(rf.score(train_df[predictors], train_df[target]))
print "OOB R^2: {:.3}".format(rf.oob_score_)

### Exercise: Plot OOB performance vs n_estimators for 5, 10, 15, ... 100 trees

### Interpretation

#### Feature Importance

Every time a split is made in any tree in the forest, that split contributes to the overall information gain of the model. We can keep track of the total information gain attributed to each feature in the model.

#### Decision Paths
http://blog.datadive.net/interpreting-random-forests/

For any one prediction, we can follow that prediction down each tree to see how each feature has contributed to that particular prediction

```
pip install treeinterpreter
```

prediction = bias + sum(contributions)

__Exercise__:

1. Find the five most expensive neighborhoods in the training data, _according to the model_
2. Tell the story of 1-2 of the neighborhoods. Why are they so expensive (according to the model).

Neighborhoods 1,3,4, and 5 are pretty similar. They all consist of really large houses in upper-class neighborhoods. Neighborhood 2 is a little different. It is much smaller, but it is expensive because of its proximity to the city center.

## Gradient Boosting

- Gradient boosting works by building one tree at a time. 
- Each successive tree is targeted on the gradient of the loss of the model up to that point. For regression, this means that each tree is targeted on the residual of the previous trees.
- If subsample < 1 and/or max_features < n_features, then it becomes stochastic gradient boosting.
- The main parameters are
    - max_depth: Also called interaction depth. This controls the bias of the individual estimators.
    - learning_rate: the weight applied to each successive tree
    - n_estimators: number of iterations.
    - max_features: the number of features available at each split

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
import pdb

gb = GradientBoostingRegressor(subsample=.7, max_depth=6, learning_rate = .05, 
                               n_estimators=500, max_features='auto')


# Split train_df
train, dev = train_test_split(train_df)

# Function to retrieve sequence of accuracies at each boosting iteration
def get_accuracy(train, dev):
    train_acc = []
    test_acc = []
    for x, y in zip(gb.staged_predict(train[predictors]), gb.staged_predict(dev[predictors])):
        train_acc.append(r2_score(x, train[target]))
        test_acc.append(r2_score(y, dev[target]))
    return train_acc, test_acc

# Get the full sequence of accuracies for each of 10-fold
kf = KFold(n_splits=10)
train_accuracies = []
test_accuracies = []
for train_idx, dev_idx in kf.split(train_df):
    train = train_df.iloc[train_idx]
    dev = train_df.iloc[dev_idx]
    gb.fit(train[predictors], train[target])
    train_acc, test_acc = get_accuracy(train, dev)
    train_accuracies.append(train_acc)
    test_accuracies.append(test_acc)
train_acc = np.mean(np.array(train_accuracies), axis=0)
test_acc = np.mean(np.array(test_accuracies), axis=0)

# Plot
d = pd.DataFrame({
        'train': train_acc,
        'test': test_acc
        })
d.plot()
plt.xlabel('Iterations')
plt.ylabel('R^2')
_ = plt.ylim([.2, 1])

### Exercise: Grid-search for best GB model

1. Use cross-validation to perform a grid search over GBDT parameters (you can choose any set of parameters)
2. Build a GBDT model on all the data using the best set of parameters
3. Plot the feature importance for the final GBDT

Best parameters

Best score

Best Model

Feature Importance

## Try all three models on the test set

In [ ]:
dt.score(test_df[predictors], test_df[target])

In [ ]:
rf.score(test_df[predictors], test_df[target])

In [ ]:
gb.score(test_df[predictors], test_df[target])